In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import glob
from IPython.display import Image

2023-05-31 19:01:39.520594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 19:01:39.621067: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-31 19:01:39.649976: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-31 19:01:40.100300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
gpu_list = [0]
physical_devices = tf.config.list_physical_devices("GPU")
final_gpu_list = [
    physical_devices[x] for x in range(len(physical_devices)) if x in gpu_list
]
tf.config.set_visible_devices(final_gpu_list, "GPU")
logical_gpus = tf.config.list_logical_devices("GPU")
strategy = tf.distribute.MirroredStrategy()
# As data and model has to be copied on all of GPUs.
REPLICAS = strategy.num_replicas_in_sync
# To copy and get data from all places we use autotune.
AUTO = tf.data.experimental.AUTOTUNE

2023-05-31 19:01:41.011193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-31 19:01:41.033537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-31 19:01:41.033668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-31 19:01:41.034413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
files = glob.glob('../../files/train/*.jpg')

In [4]:
labels = [1 if x.split('/')[-1].split('.')[0] == 'dog' else 0 for x in files]

In [32]:
def read_imgs(img, label, shape):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, (shape, shape))
    img = img / 255
    label = tf.cast(label, dtype = tf.float32)
    return img, label

def get_data(data, shape=64, repeat = True, batch = True, batch_size = 32):
    data, labels, shapes = [x[0] for x in data], [x[1] for x in data], [shape for x in range(len(data))]
    tensor = tf.data.Dataset.from_tensor_slices((data, labels, shapes))
    tensor = tensor.cache()
    tensor = tensor.map(read_imgs, num_parallel_calls=AUTO)
    if repeat:
        tensor = tensor.repeat()
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [33]:
data = [(files[x], labels[x]) for x in range(len(files))]

In [34]:
bias_init = keras.initializers.Constant(value=0.2)

![Incepv1_module](../images/incep_v1_module.png)

In [35]:
def incep_module(x,
                 filters_1x1,
                 filters_3x3_reduce,
                 filters_3x3,
                 filters_5x5_reduce,
                 filters_5x5,
                 filters_pool_proj,
                 name = None):
    # Most left in above image single 1x1 convolution
    conv_1x1 = layers.Conv2D(filters_1x1, (1, 1), padding = 'same', bias_initializer = bias_init)(x)
    
    # 1x1 and 3x3 conv from above image
    conv_3x3 = layers.Conv2D(filters_3x3_reduce, (1, 1), padding = 'same', bias_initializer = bias_init)(x)
    conv_3x3 = layers.Conv2D(filters_3x3, (3, 3), padding = 'same', bias_initializer = bias_init)(conv_3x3)

    # 1x1 and 5x5conv from above image
    conv_5x5 = layers.Conv2D(filters_5x5_reduce, (1, 1), padding = 'same', bias_initializer = bias_init)(x)
    conv_5x5 = layers.Conv2D(filters_5x5, (5, 5), padding = 'same', bias_initializer = bias_init)(conv_5x5)

    # 3x3 max pool and 1x1 conv
    pool_proj = layers.MaxPool2D((3,3), (1,1), padding = 'same')(x)
    pool_proj = layers.Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', bias_initializer=bias_init)(pool_proj)

    # return conv_1x1, conv_3x3, conv_5x5, pool_proj

    output = tf.keras.layers.concatenate([
        conv_1x1, conv_3x3, conv_5x5, pool_proj
    ], axis = 3)

    return output

In [66]:
def create_incep_model(image_size):
    input_layer = layers.Input((image_size[0], image_size[1], 3), name = 'input_layer')
    c1 = layers.Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='c1/7x7/2', bias_initializer=bias_init)(input_layer)
    max1 = layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name = 'maxpool_1')(c1)
    c2 = layers.Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='c2/1x1/2')(max1)
    c3 = layers.Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='c2/3x3/2')(c2)
    max2 = layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name = 'maxpool_2')(c3)

    # return max2
    incep1 = incep_module(max2,
                          filters_1x1=64,
                          filters_3x3_reduce=96,
                          filters_3x3=128,
                          filters_5x5_reduce=16,
                          filters_5x5=32,
                          filters_pool_proj=32,
                          name='inception_3a')
    
    incep2 = incep_module(incep1,
                          filters_1x1=128,
                          filters_3x3_reduce=128,
                          filters_3x3=192,
                          filters_5x5_reduce=32,
                          filters_5x5=96,
                          filters_pool_proj=64,
                          name='inception_3b')
    max3 = layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name = 'maxpool_3')(incep2)

    incep3 = incep_module(max3,
                          filters_1x1=192,
                          filters_3x3_reduce=96,
                          filters_3x3=208,
                          filters_5x5_reduce=16,
                          filters_5x5=48,
                          filters_pool_proj=64,
                          name='inception_4a')
    
    # we have to define auxillary outputs here
    aux1 = layers.AveragePooling2D((5, 5), strides=3)(incep3)
    aux1 = layers.Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(aux1)
    aux1 = layers.Flatten()(aux1)
    aux1 = layers.Dense(1024, activation='relu')(aux1)
    aux1 = layers.Dropout(0.7)(aux1)
    aux1 = layers.Dense(1, activation='sigmoid', name='aux_output_1')(aux1)

    incep4 = incep_module(incep3,
                          filters_1x1=160,
                          filters_3x3_reduce=112,
                          filters_3x3=224,
                          filters_5x5_reduce=24,
                          filters_5x5=64,
                          filters_pool_proj=64,
                          name='inception_4b')
    
    incep5 = incep_module(incep4,
                          filters_1x1=128,
                          filters_3x3_reduce=128,
                          filters_3x3=256,
                          filters_5x5_reduce=24,
                          filters_5x5=64,
                          filters_pool_proj=64,
                          name='inception_4c')
    
    incep6 = incep_module(incep5,
                          filters_1x1=112,
                          filters_3x3_reduce=144,
                          filters_3x3=288,
                          filters_5x5_reduce=32,
                          filters_5x5=64,
                          filters_pool_proj=64,
                          name='inception_4d')
    
    # we have to define auxillary outputs here
    aux2 = layers.AveragePooling2D((5, 5), 3)(incep6)
    aux2 = layers.Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(aux2)
    aux2 = layers.Flatten()(aux2)
    aux2 = layers.Dense(1024, activation='relu')(aux2)
    aux2 = layers.Dropout(0.7)(aux2)
    aux2 = layers.Dense(1, activation='sigmoid', name='aux_output_2')(aux2)
    
    incep7 = incep_module(incep6,
                          filters_1x1=256,
                          filters_3x3_reduce=160,
                          filters_3x3=320,
                          filters_5x5_reduce=32,
                          filters_5x5=128,
                          filters_pool_proj=128,
                          name='inception_4e')

    max4 = layers.MaxPool2D((3, 3), 2, padding='same', name = 'maxpool_4')(incep7)

    incep8 = incep_module(max4,
                          filters_1x1=256,
                          filters_3x3_reduce=160,
                          filters_3x3=320,
                          filters_5x5_reduce=32,
                          filters_5x5=128,
                          filters_pool_proj=128,
                          name='inception_5a')
    
    incep9 = incep_module(incep8,
                          filters_1x1=384,
                          filters_3x3_reduce=192,
                          filters_3x3=384,
                          filters_5x5_reduce=48,
                          filters_5x5=128,
                          filters_pool_proj=128,
                          name='inception_5b')
    
    ap = layers.GlobalAveragePooling2D(name='global_avg_pooling')(incep9)
    
    d1 = layers.Dropout(0.4)(ap)

    d2 = layers.Dense(1000, activation = 'relu')(d1)

    d3 = layers.Dense(1, activation='sigmoid')(d2)

    model = keras.Model(input_layer, [d3], name = 'incep_v1')
    
    return model

In [ ]:
tf.keras.metrics.Accuracy(thres)

In [94]:
def compile_new_model(model):
    with strategy.scope():
        loss = keras.losses.BinaryCrossentropy()
        optimizer = keras.optimizers.Adam()
        metrics = [
            keras.metrics.Accuracy()
        ]
        model.compile(
            loss = loss,
            optimizer = optimizer, 
            metrics = metrics
        )
    return model

In [95]:
with strategy.scope():
    model = create_incep_model((256, 256))
    model = compile_new_model(model)

In [96]:
model.summary()

Model: "incep_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 c1/7x7/2 (Conv2D)              (None, 128, 128, 64  9472        ['input_layer[0][0]']            
                                )                                                                 
                                                                                                  
 maxpool_1 (MaxPooling2D)       (None, 64, 64, 64)   0           ['c1/7x7/2[0][0]']               
                                                                                           

In [97]:
data_ = get_data(data, shape=256, batch_size=32)

In [90]:
model.fit(data_,
          steps_per_epoch=len(data) // (32 * REPLICAS),
          epochs = 10,
          verbose = 1)

Epoch 1/10


2023-05-31 19:47:22.096168: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 25000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:289"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT

781/781 [==============================] - 40s 47ms/step - loss: 0.7160 - accuracy: 0.0000e+00
Epoch 2/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6966 - accuracy: 0.0000e+00
Epoch 3/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6859 - accuracy: 0.0000e+00
Epoch 4/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6757 - accuracy: 0.0000e+00
Epoch 5/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6623 - accuracy: 0.0000e+00
Epoch 6/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6533 - accuracy: 0.0000e+00
Epoch 7/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6376 - accuracy: 0.0000e+00
Epoch 8/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6151 - accuracy: 0.0000e+00
Epoch 9/10
781/781 [==============================] - 37s 47ms/step - loss: 0.5789 - accuracy: 0.0000e+00
Epoch 10/10
781/781 [==============================] - 37

In [91]:
data_ = get_data(data, shape=256, batch_size=32, repeat=False)

In [92]:
preds = model.predict(data_, verbose = 1)

2023-05-31 19:53:32.836090: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 25000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:307"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT

782/782 [==============================] - 13s 15ms/step


In [93]:
preds

array([[0.906713  ],
       [0.20937087],
       [0.7852278 ],
       ...,
       [0.6930758 ],
       [0.5030427 ],
       [0.44181105]], dtype=float32)

In [98]:
# With Adam optimizer
model.fit(data_,
          steps_per_epoch=len(data) // (32 * REPLICAS),
          epochs = 10,
          verbose = 1)

Epoch 1/10


2023-05-31 19:54:15.456710: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 25000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:330"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT

781/781 [==============================] - 41s 47ms/step - loss: 254.2637 - accuracy: 0.0331
Epoch 2/10
781/781 [==============================] - 37s 47ms/step - loss: 0.7092 - accuracy: 0.0000e+00
Epoch 3/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6930 - accuracy: 0.0000e+00
Epoch 4/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6895 - accuracy: 0.0000e+00
Epoch 5/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6848 - accuracy: 0.0000e+00
Epoch 6/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6742 - accuracy: 0.0000e+00
Epoch 7/10
781/781 [==============================] - 37s 48ms/step - loss: 0.6590 - accuracy: 0.0000e+00
Epoch 8/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6567 - accuracy: 0.0000e+00
Epoch 9/10
781/781 [==============================] - 37s 47ms/step - loss: 0.6548 - accuracy: 0.0000e+00
Epoch 10/10
781/781 [==============================] - 37s 